# **Question 4**

You are required to write code to  find frequent itemsets using the above dataset (Question 1). In order to achieve the task, you are going to cover the following steps:
*	Importing required libraries
*	Creating a list from dataset (Question 1)
*	Convert list to dataframe with boolean values
*	Find frequently occurring itemsets using Apriori Algorithm
*	Find frequently occurring itemsets using F-P Growth
*	Mine the Association Rules


# Importing required libraries

In [ ]:
!pip install apyori
!pip install pyfpgrowth

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=3862d0dde88c631f00018bca6f27180e9b4c7c900940c281abb382fab15aee09
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori
     |████████████████████████████████| 1.6MB 2.8MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5479 sha256=3e225175ec9e40f186e73252b699328ae262121e21c7d75ce8db314f4110471f
  Stored in directory: /root/.cache/pip/wheels/3b/3f/0d/a04bb8b17887c1eca7d0f1a48d4aa0c09c96eb221ff7fa56c1
Successfully built pyfpgrowth


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
from apyori import apriori
import pyfpgrowth

# Creating a list from dataset

In [ ]:
data = pd.read_csv('loan_precheck_db.csv')

In [ ]:
data.head(10)

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,loan_status,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,total_bal_il,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,149521312,3025,3025,36 months,17.19%,108.14,C,C5,RENT,30000.0,Fully Paid,debt_consolidation,Debt consolidation,300xx,GA,11.88,0,5-Jul,665,669,0,9,0,2885,43.70%,22,w,0,Individual,0,0,19021,0,3,0,1,16136,3,4,1530,72.0,6600,1,1,1,5,2378.0,59.0,97.6,0,0,149,7,7,3,9.0,1,2,4,2,5,7,6,12,4,9,0,0,3,90.9,100.0,0,26381,19021,2500,19781
1,149459186,1000,1000,36 months,7.56%,31.14,A,A3,RENT,26000.0,Fully Paid,other,Other,281xx,NC,14.73,0,5-Nov,765,769,1,12,0,697,2%,23,f,0,Individual,0,0,7246,1,3,0,2,6549,1,2,186,17.0,34200,1,0,1,4,604.0,33503.0,2.0,0,0,160,2,2,0,2.0,0,5,5,9,12,6,9,17,5,12,0,0,1,100.0,0.0,0,43225,7246,34200,9025
2,149513446,18500,18500,60 months,14.74%,437.60,C,C2,RENT,144000.0,Charged Off,debt_consolidation,Debt consolidation,921xx,CA,25.48,0,Nov-75,690,694,1,20,0,62675,74.40%,40,w,0,Individual,0,0,230990,2,3,1,2,66285,2,2,19786,76.0,84200,1,0,2,4,12833.0,17275.0,77.3,0,0,520,1,1,1,1.0,0,6,10,8,12,14,16,25,10,20,0,0,3,94.7,75.0,0,326958,128960,76000,106758
3,149416464,5000,5000,36 months,20.00%,185.82,D,D2,RENT,42000.0,Fully Paid,other,Other,112xx,NY,6.46,4,7-Apr,665,669,0,4,0,1719,68.80%,8,w,0,Individual,0,0,1719,0,0,0,0,0,0,0,618,69.0,2500,0,0,0,0,430.0,781.0,68.8,0,0,143,27,27,0,27.0,0,4,4,4,7,0,4,7,4,4,0,3,0,50.0,25.0,0,2500,1719,2500,0
4,148453246,12000,12000,60 months,12.40%,269.37,B,B4,OWN,89000.0,Fully Paid,home_improvement,Home improvement,739xx,OK,17.00,0,May-99,730,734,4,11,1,8977,25.60%,21,f,0,Individual,0,0,66081,0,7,1,1,57104,1,1,5154,40.0,35000,3,3,9,2,6007.0,25523.0,26.0,0,0,145,7,7,2,7.0,0,3,3,3,4,11,4,8,3,11,0,0,2,95.2,0.0,1,114536,66081,34500,79536
5,149477700,15000,15000,36 months,17.97%,542.07,D,D1,MORTGAGE,58240.0,Fully Paid,debt_consolidation,Debt consolidation,907xx,CA,38.53,1,5-Aug,660,664,0,9,0,67035,66.40%,18,w,0,Individual,0,0,271606,0,2,1,3,34645,1,3,13753,78.0,116700,4,2,2,6,30178.0,12317.0,66.4,0,0,157,10,7,1,10.0,1,5,6,5,6,7,6,10,6,9,0,1,2,88.9,20.0,0,336009,101680,36700,39309
6,149521158,2000,2000,36 months,16.40%,70.71,C,C4,RENT,53000.0,Fully Paid,car,Car financing,765xx,TX,20.86,0,9-Sep,680,684,1,22,0,15299,34.80%,44,f,0,Individual,0,0,124465,1,15,1,1,109166,1,5,5017,89.0,44000,1,0,2,6,5658.0,13357.0,43.9,0,0,43,5,5,0,20.0,0,3,5,4,4,36,7,8,5,22,0,0,2,100.0,50.0,0,140588,124465,23800,96588
7,149089105,6500,6500,36 months,8.81%,206.13,A,A5,OWN,85000.0,Fully Paid,debt_consolidation,Debt consolidation,851xx,AZ,21.62,1,8-Feb,685,689,1,5,0,1870,33.40%,10,f,0,Individual,0,0,95903,0,3,0,3,94033,0,0,1757,67.0,5600,1,3,5,3,19181.0,3730.0,33.4,0,0,36,25,23,1,25.0,0,2,2,2,2,7,2,2,2,5,0,0,0,87.5,0.0,0,119576,95903,5600,113976
8,149484294,3000,3000,36 months,15.57%,104.84,C,C3,RENT,50000.0,Fully Paid,credit_card,Credit card refinancing,207xx,MD,17.41,0,Dec-99,680,684,0,8,0,7542,41.70%,23,w,0,Individual,0,0,25605,0,1,0,1,18063,0,1,482,61.0,18100,1,3,0,2,3201.0,7518.0,6.0,0,0,231,19,19,2,25.0,1,1,5,2,3,5,7,16,5,8,0,0,0,95.7,0.0,0,42245,25605,8000,24145
9,149104523,20000,20000,36 months,6.46%,61

In [ ]:
df = data.filter(['id','purpose','inq_last_6mths'])

# Convert list to dataframe with boolean values

In [ ]:
df.head(10)

,id,purpose,inq_last_6mths
0,149521312,debt_consolidation,0
1,149459186,other,1
2,149513446,debt_consolidation,1
3,149416464,other,0
4,148453246,home_improvement,4
5,149477700,debt_consolidation,0
6,149521158,car,1
7,149089105,debt_consolidation,1
8,149484294,credit_card,0
9,149104523,home_improvement,0


In [ ]:
user= pd.Series.unique(df.id).shape[0]
item= pd.Series.unique(df.purpose).shape[0]

In [ ]:
print('Number of total records='+str(len(df)))
print('Number of users='+str(user)+' | Loan Purpose ='+str(item))

Number of total records=75327
Number of users=75327 | Loan Purpose =12


In [ ]:
df.purpose.unique()

array(['debt_consolidation', 'other', 'home_improvement', 'car',
       'credit_card', 'major_purchase', 'small_business', 'moving',
       'house', 'medical', 'vacation', 'renewable_energy'], dtype=object)

# Find frequently occurring itemsets using Apriori Algorithm

In [ ]:
transactions = df.groupby(['id'])['purpose'].apply(list)
mylist = transactions.values.tolist()

#first 3
print(mylist[:3])

[['debt_consolidation'], ['debt_consolidation'], ['debt_consolidation']]


In [ ]:
records =(df.groupby(['id','purpose']).size().unstack().reset_index().fillna(0).set_index('id')) 
records.iloc[0:10,:]

purpose,car,credit_card,debt_consolidation,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation
id,,,,,,,,,,,,
138244662,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142842779,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142912147,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143046459,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143185276,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143491446,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143521353,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143579616,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143946285,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Find frequently occurring itemsets using F-P Growth

In [ ]:
patterns = pyfpgrowth.find_frequent_patterns(transactions, 2)

In [ ]:
patterns

{('car',): 861,
 ('credit_card',): 17131,
 ('debt_consolidation',): 40768,
 ('home_improvement',): 5583,
 ('house',): 872,
 ('major_purchase',): 1756,
 ('medical',): 1090,
 ('moving',): 597,
 ('other',): 5288,
 ('renewable_energy',): 43,
 ('small_business',): 610,
 ('vacation',): 728}

# Mine the Association Rules

In [ ]:
association_rules = apriori(mylist,min_support=0.05)

In [ ]:
association_results = list(association_rules)

In [ ]:
results = pd.DataFrame(association_results)

results.head()

,items,support,ordered_statistics
0,(credit_card),0.227422,"[((), (credit_card), 0.22742177439696257, 1.0)]"
1,(debt_consolidation),0.541214,"[((), (debt_consolidation), 0.5412136418548462..."
2,(home_improvement),0.074117,"[((), (home_improvement), 0.07411685053168186,..."
3,(other),0.070201,"[((), (other), 0.0702005920851753, 1.0)]"


In [ ]:
for i in results:
 
 def pre_confidence(i):
   confidence = i[0].confidence
   return confidence
  
 def pre_lift(i):
   lift = i[0].lift
   return lift
 
 results['Items'] = results ['items']
 results['Support'] = results ['support']
 results['Confidence'] = results['ordered_statistics'].apply(pre_confidence)
 results['Lift'] = results['ordered_statistics'].apply(pre_lift)
 
 output = pd.DataFrame(results)
 output.drop(["items","support","ordered_statistics"], axis = 1, inplace = True) 

output

,Items,Support,Confidence,Lift
0,(credit_card),0.227422,0.227422,1.0
1,(debt_consolidation),0.541214,0.541214,1.0
2,(home_improvement),0.074117,0.074117,1.0
3,(other),0.070201,0.070201,1.0


End of Question